In [33]:
import boto3
import json

In [30]:
from os import path
from utils import Utils

In [45]:
LAMBDA_ACCESS_POLICY_ARN = 'arn:aws:iam::059062539581:policy/LambdaS3AccessPolicy'
LAMBDA_ROLE = 'Lambda_Execution_Role'
LAMBDA_ROLE_ARN = 'arn:aws:iam::059062539581:role/Lambda_Execution_Role'
LAMBDA_TIMEOUT = 10
LAMBDA_MEMORY = 128
LAMBDA_HANDLER = 'lambda_function.handler'
PYTHON_36_RUNTIME = 'python3.6'
NODEJS_810_RUNTIME = 'nodejs10.x'
JAVA_8_RUNTIME = 'java8'
NODEJS_LAMBDA_NAME = 'NodeJSLambdaFunction'
PYTHON_LAMBDA_NAME = 'PythonLambdaFunction'
JAVA_LAMBDA_NAME = 'JavaLambdaFunction'

### Create boto3 aws client

In [38]:
def lambda_client():
    aws_lambda = boto3.client('lambda', region_name='eu-west-1')
    """ :type : pyboto3.lambda """
    return aws_lambda

### Create aws iam cient to create role and access policy

In [12]:
def iam_client():
    iam = boto3.client('iam')
    """ :type : pyboto3.iam """
    return iam


### Creating an IAM Lambda Access Policy

In [13]:
def create_access_policy_for_lambda():
    s3_access_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Action": [
                    "s3:*",
                    "logs:CreateLogGroup",
                    "logs:CreateLogStream",
                    "logs:PutLogEvents"
                ],
                "Effect": "Allow",
                "Resource": "*"
            }
        ]
    }

    return iam_client().create_policy(
        PolicyName='LambdaS3AccessPolicy',
        PolicyDocument=json.dumps(s3_access_policy_document),
        Description='Allows lambda function to access S3 resources'
    )

### Creating an IAM Execution Role

In [14]:
def create_execution_role_for_lambda():
    lamda_execution_assumption_role = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    return iam_client().create_role(
        RoleName=LAMBDA_ROLE,
        AssumeRolePolicyDocument=json.dumps(lamda_execution_assumption_role),
        Description="Gives necessary permissions for lambda to be executed"
    )

### Attaching IAM Access Policy to IAM Execution Role

In [15]:
def attach_access_policy_to_execution_role():
    return iam_client().attach_role_policy(
        RoleName=LAMBDA_ROLE,
        PolicyArn=LAMBDA_ACCESS_POLICY_ARN
    )

### Implementing Utility Methods for Source Code Bytes Reading

In [16]:
cat utils.py

import os
import io

from zipfile import ZipFile

class Utils:
    @staticmethod
    def make_zip_file_bytes(path):
        buf = io.BytesIO()
        with ZipFile(buf, 'w') as z:
            for full_path, archive_name in Utils.files_to_zip(path=path):
                z.write(full_path, archive_name)
        return buf.getvalue()

    @staticmethod
    def files_to_zip(path):
        for root, dirs, files in os.walk(path):
            for f in files:
                full_path = os.path.join(root, f)
                archive_name = full_path[len(path) + len(os.sep):]
                yield full_path, archive_name

    @staticmethod
    def read_jar_file(path):
        with open(path, 'rb') as binary_file:
            data = binary_file.read()
        return data

### Developing a Python Lambda Function

In [17]:
cat python_lambda_function.py

cat: python_lambda_function.py: No such file or directory


### Deploying Python Lambda Function

In [35]:
def deploy_lambda_function(function_name, runtime, handler, role_arn, source_folder):
    print (f"source_folder: {source_folder}")
    print (f"function_name: {function_name}")
    print (f"role_arn: {role_arn}")
    print (f"runtime: {runtime}")
    
    folder_path = path.join(path.abspath("."), source_folder)

    print (f"folder_path: {folder_path}")
    
    if runtime is not JAVA_8_RUNTIME:
        zip_file = Utils.make_zip_file_bytes(path=folder_path)
    else:
        zip_file = Utils.read_jar_file(folder_path)

    print (f"zip_file: {zip_file}")
    
    return lambda_client().create_function(
        FunctionName=function_name,
        Runtime=runtime,
        Role=role_arn,
        Handler=handler,
        Code={
            'ZipFile': zip_file
        },
        Timeout=LAMBDA_TIMEOUT,
        MemorySize=LAMBDA_MEMORY,
        Publish=False
    )


### AWS Console Checkpoint: Python Lambda Function
 - Open AWS Console and verify that lambda function is created as expected

In [19]:
def invoke_lambda_function(function_name):
    return lambda_client().invoke(FunctionName=function_name)


### Modify lambda function configuration

In [20]:
def add_environment_variables_to_lambda(function_name, variables):
    return lambda_client().update_function_configuration(
        FunctionName=function_name,
        Environment=variables
    )


### Modify lambda function code

In [53]:
def update_lambda_function_code(function_name, source_folder):
    folder_path = path.join(path.abspath('.'), source_folder)
    zip_file = Utils.make_zip_file_bytes(path=folder_path)

    return lambda_client().update_function_code(
        FunctionName=function_name,
        ZipFile=zip_file
    )

### Publish new version of lambda

In [22]:
def publish_a_new_version(function_name):
    return lambda_client().publish_version(
        FunctionName=function_name
    )


In [23]:
def create_alias_for_new_version(function_name, alias_name, version):
    return lambda_client().create_alias(
        FunctionName=function_name,
        Name=alias_name,
        FunctionVersion=version,
        Description='This is the ' + alias_name + ' alias for function'
    )

In [24]:
def invoke_lambda_with_alias(function_name, alias_name):
    return lambda_client().invoke(
        FunctionName=function_name,
        Qualifier=alias_name
    )

In [25]:
def get_function(function_name):
    return lambda_client().get_function(FunctionName=function_name)

In [26]:
def get_all_functions():
    return lambda_client().list_functions()

In [27]:
def increase_lambda_execution_memory(function_name, new_memory_size):
    return lambda_client().update_function_configuration(
        FunctionName=function_name,
        MemorySize=new_memory_size
    )

In [28]:
def delete_lambda_function(function_name):
    return lambda_client().delete_function(FunctionName=function_name)

In [64]:
# print(create_access_policy_for_lambda())


In [65]:
# print(create_execution_role_for_lambda())


In [66]:
# print(attach_access_policy_to_execution_role())


In [67]:
# print(deploy_lambda_function(PYTHON_LAMBDA_NAME, PYTHON_36_RUNTIME, LAMBDA_HANDLER, LAMBDA_ROLE_ARN, 'python_lambda'))


In [68]:
# print(deploy_lambda_function(NODEJS_LAMBDA_NAME, NODEJS_810_RUNTIME, LAMBDA_HANDLER, LAMBDA_ROLE_ARN, 'nodejs_lambda'))


In [69]:
# print(deploy_lambda_function(JAVA_LAMBDA_NAME, JAVA_8_RUNTIME, 'com.amazonaws.lambda.demo.LambdaFunctionHandler::handleRequest', LAMBDA_ROLE_ARN, 'JavaLambdaFunction/target/demo-1.0.0.jar'))


In [70]:
# response = invoke_lambda_function(PYTHON_LAMBDA_NAME)


In [71]:
#print(response['Payload'].read().decode())


In [72]:
env_variables = {
    'Variables': {
        'ENV_VAR_TEST': 'This is an environment variable!'
    }
}


In [73]:
#add_environment_variables_to_lambda(PYTHON_LAMBDA_NAME, env_variables)


In [74]:
#print(update_lambda_function_code(PYTHON_LAMBDA_NAME, 'python_lambda'))


In [75]:
response = invoke_lambda_function(PYTHON_LAMBDA_NAME)
print(response['Payload'].read().decode())


{"statusCode": 200, "message": "This is an environment variable!"}


In [76]:
print(publish_a_new_version(PYTHON_LAMBDA_NAME))


{'ResponseMetadata': {'RequestId': 'e2717c3c-5902-4f1e-a03a-a3e47c5aa836', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Wed, 22 Sep 2021 02:38:26 GMT', 'content-type': 'application/json', 'content-length': '980', 'connection': 'keep-alive', 'x-amzn-requestid': 'e2717c3c-5902-4f1e-a03a-a3e47c5aa836'}, 'RetryAttempts': 0}, 'FunctionName': 'PythonLambdaFunction', 'FunctionArn': 'arn:aws:lambda:eu-west-1:059062539581:function:PythonLambdaFunction:2', 'Runtime': 'python3.6', 'Role': 'arn:aws:iam::059062539581:role/Lambda_Execution_Role', 'Handler': 'lambda_function.handler', 'CodeSize': 396, 'Description': '', 'Timeout': 10, 'MemorySize': 256, 'LastModified': '2021-09-22T02:37:20.817+0000', 'CodeSha256': 'X25DSxSpPVCvDbgCVRkoyLAMKEQC/4b3byECA0lpF78=', 'Version': '2', 'Environment': {'Variables': {'ENV_VAR_TEST': 'This is an environment variable!'}}, 'TracingConfig': {'Mode': 'PassThrough'}, 'RevisionId': '8182ceef-4772-42c3-b15c-0a2607a243b1', 'State': 'Active', 'LastUpdateStatus': 'Succ

In [77]:
create_alias_for_new_version(PYTHON_LAMBDA_NAME, 'PROD', '1')


ResourceConflictException: An error occurred (ResourceConflictException) when calling the CreateAlias operation: Alias already exists: arn:aws:lambda:eu-west-1:059062539581:function:PythonLambdaFunction:PROD

In [78]:
response = invoke_lambda_with_alias(PYTHON_LAMBDA_NAME, 'PROD')
print(response['Payload'].read().decode())


{"statusCode": 200, "message": "This is an environment variable!"}


In [79]:
print(get_function(PYTHON_LAMBDA_NAME))


{'ResponseMetadata': {'RequestId': '05d787a2-8be0-4984-a385-cb8f9f010a2d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 22 Sep 2021 02:38:41 GMT', 'content-type': 'application/json', 'content-length': '2744', 'connection': 'keep-alive', 'x-amzn-requestid': '05d787a2-8be0-4984-a385-cb8f9f010a2d'}, 'RetryAttempts': 0}, 'Configuration': {'FunctionName': 'PythonLambdaFunction', 'FunctionArn': 'arn:aws:lambda:eu-west-1:059062539581:function:PythonLambdaFunction', 'Runtime': 'python3.6', 'Role': 'arn:aws:iam::059062539581:role/Lambda_Execution_Role', 'Handler': 'lambda_function.handler', 'CodeSize': 396, 'Description': '', 'Timeout': 10, 'MemorySize': 256, 'LastModified': '2021-09-22T02:37:20.817+0000', 'CodeSha256': 'X25DSxSpPVCvDbgCVRkoyLAMKEQC/4b3byECA0lpF78=', 'Version': '$LATEST', 'Environment': {'Variables': {'ENV_VAR_TEST': 'This is an environment variable!'}}, 'TracingConfig': {'Mode': 'PassThrough'}, 'RevisionId': 'ca849d2c-8343-43ac-ae6f-dcd1f92cee74', 'State': 'Active', 'L

In [80]:
print(get_all_functions())


{'ResponseMetadata': {'RequestId': 'b18adcbe-3e9d-4e65-95a4-5b70a976cba3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 22 Sep 2021 02:38:45 GMT', 'content-type': 'application/json', 'content-length': '1941', 'connection': 'keep-alive', 'x-amzn-requestid': 'b18adcbe-3e9d-4e65-95a4-5b70a976cba3'}, 'RetryAttempts': 0}, 'Functions': [{'FunctionName': 'PythonLambdaFunction', 'FunctionArn': 'arn:aws:lambda:eu-west-1:059062539581:function:PythonLambdaFunction', 'Runtime': 'python3.6', 'Role': 'arn:aws:iam::059062539581:role/Lambda_Execution_Role', 'Handler': 'lambda_function.handler', 'CodeSize': 396, 'Description': '', 'Timeout': 10, 'MemorySize': 256, 'LastModified': '2021-09-22T02:37:20.817+0000', 'CodeSha256': 'X25DSxSpPVCvDbgCVRkoyLAMKEQC/4b3byECA0lpF78=', 'Version': '$LATEST', 'Environment': {'Variables': {'ENV_VAR_TEST': 'This is an environment variable!'}}, 'TracingConfig': {'Mode': 'PassThrough'}, 'RevisionId': 'ca849d2c-8343-43ac-ae6f-dcd1f92cee74', 'PackageType': 'Zip'}, {

In [81]:
increase_lambda_execution_memory(PYTHON_LAMBDA_NAME, 256)


{'ResponseMetadata': {'RequestId': '0c98a649-dcbc-440b-a656-fc907c153a0f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 22 Sep 2021 02:38:51 GMT',
   'content-type': 'application/json',
   'content-length': '984',
   'connection': 'keep-alive',
   'x-amzn-requestid': '0c98a649-dcbc-440b-a656-fc907c153a0f'},
  'RetryAttempts': 0},
 'FunctionName': 'PythonLambdaFunction',
 'FunctionArn': 'arn:aws:lambda:eu-west-1:059062539581:function:PythonLambdaFunction',
 'Runtime': 'python3.6',
 'Role': 'arn:aws:iam::059062539581:role/Lambda_Execution_Role',
 'Handler': 'lambda_function.handler',
 'CodeSize': 396,
 'Description': '',
 'Timeout': 10,
 'MemorySize': 256,
 'LastModified': '2021-09-22T02:38:51.174+0000',
 'CodeSha256': 'X25DSxSpPVCvDbgCVRkoyLAMKEQC/4b3byECA0lpF78=',
 'Version': '$LATEST',
 'Environment': {'Variables': {'ENV_VAR_TEST': 'This is an environment variable!'}},
 'TracingConfig': {'Mode': 'PassThrough'},
 'RevisionId': '07d72ddd-5fa8-40b3-bb3d-7343f8d621a8',
 'State

In [82]:
delete_lambda_function(NODEJS_LAMBDA_NAME)

ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the DeleteFunction operation: Function not found: arn:aws:lambda:eu-west-1:059062539581:function:NodeJSLambdaFunction